In [ ]:
#!/usr/bin/env python3
# ────────────────────────────────────────────────────────────────
#  TSMAE  –  single-cell NumPy reference
#    · Generate dummy data / weights
#    · Define LSTM_ENCODER, MEMORY_ADDRESS, LSTM_DECODER
#    · Run forward-pass and print results
# ────────────────────────────────────────────────────────────────
import numpy as np
from pathlib import Path

# ─────────────── HYPER-PARAMETERS (MATCH VERILOG) ───────────────
DATA_WIDTH      = 32
FRACT_WIDTH     = 24
INPUT_SIZE      = 1
HIDDEN_SIZE     = 10
MEMORY_SIZE     = 10
SEQ_LEN         = 10
SCALE_WTS       = 0.05
rng             = np.random.default_rng(42)
OUTDIR          = Path(".")
SCALE           = 1 << FRACT_WIDTH
# ----------------------------------------------------------------
sigmoid = lambda x: 1.0 / (1.0 + np.exp(-x))

def float_to_fixed(x, int_bits=8, fract_bits=24):
    """Convert float to 8.24 signed fixed-point integer."""
    scale = 2 ** fract_bits
    max_val = 2 ** (int_bits - 1) - 1 / scale
    min_val = -2 ** (int_bits - 1)
    x_clipped = np.clip(x, min_val, max_val)
    return np.round(x_clipped * scale).astype(np.int32)

def fixed_to_hex(x, int_bits=8, fract_bits=24):
    """Convert signed Qm.n fixed-point to 8-char hex (2's complement)."""
    x = x.astype(np.int32)
    x = x.view(np.uint32)  # reinterpret bits as unsigned
    return f"{x:08x}"


def save_mem(name, arr, int_bits=8, fract_bits=24):
    """Save array as hex fixed-point .MEM file."""
    fixed = float_to_fixed(arr, int_bits, fract_bits)
    hex_vals = [fixed_to_hex(val, int_bits, fract_bits) for val in fixed.flatten()]
    with open(OUTDIR / name, 'w') as f:
        for val in hex_vals:
            f.write(val + '\n')

def save_txt(name, arr, fmt="%.8f"):
    np.savetxt(OUTDIR / name, arr.astype(np.float32), fmt=fmt)

def minmax_normalise(x):
    col_min = x.min(axis=0, keepdims=True)
    col_max = x.max(axis=0, keepdims=True)
    return (x - col_min) / np.where(col_max == col_min, 1.0, col_max - col_min)

def print_variable(name, arr, indent=0):
    """Print array elements vertically in float and int formats."""
    print(f"\n{name}:")
    fixed = float_to_fixed(arr, 8, 24)
    for i, (f_val, i_val) in enumerate(zip(arr.flatten(), fixed.flatten())):
        hex_val = fixed_to_hex(i_val, 8, 24)
        print(f"{'  '*indent}{name}[{i}] = {f_val:.8f} (int: {i_val}, hex: {hex_val})")

def concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh):
    """Helper function for LSTM matrix operations."""
    W_x = W_ih @ x_t
    W_h = W_hh @ h_prev
    return W_x + W_h + b_ih + b_hh

def dec_concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh):

    W_x = W_ih @ x_t
    W_h = W_hh @ h_prev
    return W_x + W_h + b_ih + b_hh

# ──────────────────── INPUT_GENERATOR ───────────────────────────
# Generate sensor window x (normalized)
raw = rng.normal(0, 1, size=(SEQ_LEN, INPUT_SIZE)).astype(np.float32)
x = minmax_normalise(raw)
save_txt("input_window.txt", x)
save_mem("input_window.mem", x)
print_variable("x", x)

# Generate encoder weights
W_ih = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE, INPUT_SIZE)).astype(np.float32)
W_hh = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE, HIDDEN_SIZE)).astype(np.float32)
b_ih = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE,)).astype(np.float32)
b_hh = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE,)).astype(np.float32)
save_txt("W_ih.txt", W_ih); save_txt("W_hh.txt", W_hh)
save_txt("b_ih.txt", b_ih); save_txt("b_hh.txt", b_hh)
save_mem("W_ih.mem", W_ih); save_mem("W_hh.mem", W_hh)
save_mem("b_ih.mem", b_ih); save_mem("b_hh.mem", b_hh)

W_big = np.concatenate([W_ih, W_hh], axis=1)
b_big = b_ih + b_hh
save_txt("encoder_W.txt", W_big); save_txt("encoder_b.txt", b_big)
save_mem("encoder_W.mem", W_big); save_mem("encoder_b.mem", b_big)

# Generate memory items
memory = rng.normal(0, SCALE_WTS, (MEMORY_SIZE, HIDDEN_SIZE)).astype(np.float32)
save_txt("memory.txt", memory)
save_mem("memory.mem", memory)

# Generate decoder weights
dec_W_ih = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE, HIDDEN_SIZE)).astype(np.float32)  # Adjusted for z_hat input
dec_W_hh = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE, HIDDEN_SIZE)).astype(np.float32)
dec_b_ih = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE,)).astype(np.float32)
dec_b_hh = rng.normal(0, SCALE_WTS, (4*HIDDEN_SIZE,)).astype(np.float32)
dec_W_out = rng.normal(0, SCALE_WTS, (INPUT_SIZE, HIDDEN_SIZE)).astype(np.float32)
dec_b_out = rng.normal(0, SCALE_WTS, (INPUT_SIZE,)).astype(np.float32)
save_txt("dec_W_ih.txt", dec_W_ih); save_txt("dec_W_hh.txt", dec_W_hh)
save_txt("dec_b_ih.txt", dec_b_ih); save_txt("dec_b_hh.txt", dec_b_hh)
save_txt("dec_W_out.txt", dec_W_out); save_txt("dec_b_out.txt", dec_b_out)
save_mem("dec_W_ih.mem", dec_W_ih); save_mem("dec_W_hh.mem", dec_W_hh)
save_mem("dec_b_ih.mem", dec_b_ih); save_mem("dec_b_hh.mem", dec_b_hh)
save_mem("dec_W_out.mem", dec_W_out); save_mem("dec_b_out.mem", dec_b_out)

print("✅  Dummy weights / inputs generated\n")

# ─────────────────────── LSTM_ENCODER ───────────────────────────
def lstm_cell_enc(x_t, h_prev, c_prev, W_ih, W_hh, b_ih, b_hh):
    """LSTM cell for encoder."""
    idx_i = slice(0, HIDDEN_SIZE)
    idx_f = slice(HIDDEN_SIZE, 2*HIDDEN_SIZE)
    idx_g = slice(2*HIDDEN_SIZE, 3*HIDDEN_SIZE)
    idx_o = slice(3*HIDDEN_SIZE, 4*HIDDEN_SIZE)

    gates = concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh)

    i_raw = gates[idx_i]
    f_raw = gates[idx_f]
    g_raw = gates[idx_g]
    o_raw = gates[idx_o]

    i_t = sigmoid(i_raw)
    f_t = sigmoid(f_raw)
    g_t = np.tanh(g_raw)
    o_t = sigmoid(o_raw)

    c_t = f_t * c_prev + i_t * g_t
    h_t = o_t * np.tanh(c_t)

    def r(arr): return [round(v, 10) for v in arr.tolist()]
    print(f"\n{'='*60}\nEncoder LSTM Cell")
    print(f"x_t             : {r(x_t)}")
    print(f"h_prev          : {r(h_prev)}")
    print(f"c_prev          : {r(c_prev)}")
    print(f"i_raw (pre-sigm): {r(i_raw)}")
    print(f"f_raw (pre-sigm): {r(f_raw)}")
    print(f"g_raw (pre-tanh): {r(g_raw)}")
    print(f"o_raw (pre-sigm): {r(o_raw)}")
    print(f"i_t (sigmoid)   : {r(i_t)}")
    print(f"f_t (sigmoid)   : {r(f_t)}")
    print(f"g_t (tanh)      : {r(g_t)}")
    print(f"o_t (sigmoid)   : {r(o_t)}")
    print(f"c_t (cell state): {r(c_t)}")
    print(f"h_t (output)    : {r(h_t)}")

    return h_t, c_t

def lstm_encoder_forward(x, W_ih, W_hh, b_ih, b_hh):
    """
    x : (SEQ_LEN, INPUT_SIZE)
    returns z : (HIDDEN_SIZE,)
    """
    h_t = np.zeros(HIDDEN_SIZE, np.float32)
    c_t = np.zeros(HIDDEN_SIZE, np.float32)

    for t in range(SEQ_LEN):
        h_t, c_t = lstm_cell_enc(x[t], h_t, c_t, W_ih, W_hh, b_ih, b_hh)

    z = h_t.astype(np.float32)
    print_variable("z", z)
    return z

# ───────────────────── MEMORY_ADDRESS ───────────────────────────
def memory_address(z, memory_items, sparsity_threshold=0.05, eps=1e-8):
    sim = z @ memory_items.T  # sim[i]
    exp_sim = np.exp(sim - sim.max())  # exp_sim[i]
    sum_exp_sim = np.sum(exp_sim)      # scalar
    q_softmax = exp_sim / sum_exp_sim  # q_softmax[i]
    q_rectified = np.maximum(q_softmax - sparsity_threshold, 0.0)  # q_rectified[i]
    sum_q_rectified = np.sum(q_rectified)  # scalar
    q_internal = q_rectified / max(sum_q_rectified, eps)  # q_internal[i]
    z_hat_internal = q_internal @ memory_items  # z_hat_internal[j]

    # === Debug printing (Verilog signal names) ===
    print_variable("z", z)
    print_variable("sim", sim)
    print_variable("exp_sim", exp_sim)
    print(f"\nsum_exp_sim = {sum_exp_sim:.8f}")
    print_variable("q_softmax", q_softmax)
    print_variable("q_rectified", q_rectified)
    print(f"\nsum_q_rectified = {sum_q_rectified:.8f}")
    print_variable("q_internal", q_internal)
    print_variable("z_hat_internal", z_hat_internal)

    # Return values
    return z_hat_internal.astype(np.float32), q_internal.astype(np.float32)


# ────────────────────── LSTM_DECODER ────────────────────────────
def lstm_cell_dec(x_t, h_prev, c_prev, W_ih, W_hh, b_ih, b_hh):
    """LSTM cell for decoder."""
    idx_i = slice(0, HIDDEN_SIZE)
    idx_f = slice(HIDDEN_SIZE, 2*HIDDEN_SIZE)
    idx_g = slice(2*HIDDEN_SIZE, 3*HIDDEN_SIZE)
    idx_o = slice(3*HIDDEN_SIZE, 4*HIDDEN_SIZE)

    gates = dec_concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh)

    i_raw = gates[idx_i]
    f_raw = gates[idx_f]
    g_raw = gates[idx_g]
    o_raw = gates[idx_o]

    i_t = sigmoid(i_raw)
    f_t = sigmoid(f_raw)
    g_t = np.tanh(g_raw)
    o_t = sigmoid(o_raw)

    c_t = f_t * c_prev + i_t * g_t
    h_t = o_t * np.tanh(c_t)

    def r(arr): return [round(v, 10) for v in arr.tolist()]
    print(f"\n{'='*60}\nDecoder LSTM Cell")
    print(f"x_t             : {r(x_t)}")
    print(f"h_prev          : {r(h_prev)}")
    print(f"c_prev          : {r(c_prev)}")
    print(f"i_raw (pre-sigm): {r(i_raw)}")
    print(f"f_raw (pre-sigm): {r(f_raw)}")
    print(f"g_raw (pre-tanh): {r(g_raw)}")
    print(f"o_raw (pre-sigm): {r(o_raw)}")
    print(f"i_t (sigmoid)   : {r(i_t)}")
    print(f"f_t (sigmoid)   : {r(f_t)}")
    print(f"g_t (tanh)      : {r(g_t)}")
    print(f"o_t (sigmoid)   : {r(o_t)}")
    print(f"c_t (cell state): {r(c_t)}")
    print(f"h_t (output)    : {r(h_t)}")

    return h_t, c_t

def lstm_decoder_forward(z_hat, seq_len=SEQ_LEN):
    W_ih = np.loadtxt("dec_W_ih.txt", np.float32).reshape(4*HIDDEN_SIZE, HIDDEN_SIZE)
    W_hh = np.loadtxt("dec_W_hh.txt", np.float32).reshape(4*HIDDEN_SIZE, HIDDEN_SIZE)
    b_ih = np.loadtxt("dec_b_ih.txt", np.float32)
    b_hh = np.loadtxt("dec_b_hh.txt", np.float32)
    W_out = np.loadtxt("dec_W_out.txt", np.float32).reshape(INPUT_SIZE, HIDDEN_SIZE)
    b_out = np.loadtxt("dec_b_out.txt", np.float32)

    # Repeat z_hat to form z_hat_seq ∈ ℝ(10×10)
    z_hat_seq = np.repeat(z_hat[np.newaxis, :], seq_len, axis=0)  # Shape: (10, 10)

    h_t = np.zeros(HIDDEN_SIZE, np.float32)  # Initialize with zeros instead of z_hat
    c_t = np.zeros(HIDDEN_SIZE, np.float32)
    x_recon = np.zeros((seq_len, INPUT_SIZE), np.float32)

    for t in range(seq_len):
        x_in = z_hat_seq[t]  # Use z_hat_seq[t] ∈ ℝ(10) as input
        h_t, c_t = lstm_cell_dec(x_in, h_t, c_t, W_ih, W_hh, b_ih, b_hh)
        x_recon[t] = W_out @ h_t + b_out
    print_variable("x_recon", x_recon)
    return x_recon

# ─────────────────────── TSMAE_TOP ──────────────────────────────
def tsmae_forward(x, memory_items, sparsity_threshold=0.05):
    W_ih = np.loadtxt("W_ih.txt", np.float32).reshape(4*HIDDEN_SIZE, INPUT_SIZE)
    W_hh = np.loadtxt("W_hh.txt", np.float32).reshape(4*HIDDEN_SIZE, HIDDEN_SIZE)
    b_ih = np.loadtxt("b_ih.txt", np.float32)
    b_hh = np.loadtxt("b_hh.txt", np.float32)

    # LSTM_ENCODER
    z = lstm_encoder_forward(x, W_ih, W_hh, b_ih, b_hh)

    # MEMORY_ADDRESS
    z_hat, q = memory_address(z, memory_items, sparsity_threshold)

    # LSTM_DECODER
    x_recon = lstm_decoder_forward(z_hat, seq_len=SEQ_LEN)

    # L2_SCORE
    loss_rec = np.linalg.norm(x - x_recon)
    print(f"\nL2 reconstruction error: {loss_rec:.6f}")
    return x_recon, q, z, z_hat, loss_rec

# ─────────────────────── SIMULATION ─────────────────────────────
if __name__ == "__main__":
    x_recon, q, z, z_hat, loss_rec = tsmae_forward(x, memory)


x:
x[0] = 0.78010529 (int: 13087995, hex: 00c7b4fb)
x[1] = 0.31506816 (int: 5285966, hex: 0050a84e)
x[2] = 0.93425316 (int: 15674167, hex: 00ef2b37)
x[3] = 1.00000000 (int: 16777216, hex: 01000000)
x[4] = 0.00000000 (int: 0, hex: 00000000)
x[5] = 0.22439331 (int: 3764695, hex: 003971d7)
x[6] = 0.71893609 (int: 12061746, hex: 00b80c32)
x[7] = 0.56535918 (int: 9485153, hex: 0090bb61)
x[8] = 0.66891479 (int: 11222528, hex: 00ab3e00)
x[9] = 0.37971756 (int: 6370604, hex: 0061352c)
✅  Dummy weights / inputs generated


Encoder LSTM Cell
x_t             : [0.7801052928]
h_prev          : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
c_prev          : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
i_raw (pre-sigm): [0.0820086226, 0.0661033541, 0.085813731, 0.1705156565, 0.0062707365, 0.0212699659, -0.0233750716, -0.0964648873, -0.0486228988, -0.0779935569]
f_raw (pre-sigm): [0.076677911, -0.0234690309, -0.0677668452, -0.184801802, -0.2208026946, -0.063843593, 0.1271240264, 0.1328600

In [ ]:
def q8_24_to_float(hex_list):
    results = []
    for hex_str in hex_list:
        val = int(hex_str, 16)
        if val & (1 << 31):  # Nếu là số âm (bit dấu = 1)
            val -= (1 << 32)
        float_val = val / (1 << 24)
        results.append(float_val)
    return results

# === Ví dụ sử dụng ===
hex_values = [
    "fff5ed39",
    "fffd2ca4",
    "00028912",
    "fffc4bdf",
    "fffd13e1",
    "ffffc674",
    "ffff58f8",
    "00020c3d",
    "fff916e7",
    "fffefdfe"
]



float_values = q8_24_to_float(hex_values)

for i, (h, f) in enumerate(zip(hex_values, float_values)):
    print(f"q_softmax[{i}] = 0x{h} = {f:.6f}")


q_softmax[0] = 0xfff5ed39 = -0.039349
q_softmax[1] = 0xfffd2ca4 = -0.011038
q_softmax[2] = 0x00028912 = 0.009904
q_softmax[3] = 0xfffc4bdf = -0.014467
q_softmax[4] = 0xfffd13e1 = -0.011415
q_softmax[5] = 0xffffc674 = -0.000878
q_softmax[6] = 0xffff58f8 = -0.002549
q_softmax[7] = 0x00020c3d = 0.007999
q_softmax[8] = 0xfff916e7 = -0.026994
q_softmax[9] = 0xfffefdfe = -0.003937


In [ ]:
import numpy as np

Q = 24
scale = 1 << Q

def to_fixed(x):
    return np.round(x * scale).astype(np.int32)

def from_fixed(x):
    return x.astype(np.float32) / scale

def dec_concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh):
    return W_ih @ x_t + W_hh @ h_prev + b_ih + b_hh

def run_testcase(testcase_id, x_t, h_prev, W_ih, W_hh, b_ih, b_hh):
    result = dec_concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh)
    print(f"\n>> Python Testcase {testcase_id} output:")
    for i, val in enumerate(result):
        print(f"out[{i}] = {val:.6f}")

def main():
    INPUT_SIZE = 10
    HIDDEN_SIZE = 10

    # === Testcase 1 ===
    x_t = np.ones(INPUT_SIZE, dtype=np.float32) * 1.0
    h_prev = np.ones(HIDDEN_SIZE, dtype=np.float32) * 0.5
    W_ih = np.ones((HIDDEN_SIZE, INPUT_SIZE), dtype=np.float32) * 1.0
    W_hh = np.ones((HIDDEN_SIZE, HIDDEN_SIZE), dtype=np.float32) * 0.5
    b_ih = np.ones(HIDDEN_SIZE, dtype=np.float32) * 0.125
    b_hh = np.zeros(HIDDEN_SIZE, dtype=np.float32)
    run_testcase(1, x_t, h_prev, W_ih, W_hh, b_ih, b_hh)

    # === Testcase 2 ===
    x_t = np.ones(INPUT_SIZE, dtype=np.float32) * -1.0
    h_prev = np.ones(HIDDEN_SIZE, dtype=np.float32) * 1.0
    W_ih = np.ones((HIDDEN_SIZE, INPUT_SIZE), dtype=np.float32) * 2.0
    W_hh = np.ones((HIDDEN_SIZE, HIDDEN_SIZE), dtype=np.float32) * 1.0
    b_ih = np.zeros(HIDDEN_SIZE, dtype=np.float32)
    b_hh = np.zeros(HIDDEN_SIZE, dtype=np.float32)
    run_testcase(2, x_t, h_prev, W_ih, W_hh, b_ih, b_hh)

if __name__ == "__main__":
    main()



>> Python Testcase 1 output:
out[0] = 12.625000
out[1] = 12.625000
out[2] = 12.625000
out[3] = 12.625000
out[4] = 12.625000
out[5] = 12.625000
out[6] = 12.625000
out[7] = 12.625000
out[8] = 12.625000
out[9] = 12.625000

>> Python Testcase 2 output:
out[0] = -10.000000
out[1] = -10.000000
out[2] = -10.000000
out[3] = -10.000000
out[4] = -10.000000
out[5] = -10.000000
out[6] = -10.000000
out[7] = -10.000000
out[8] = -10.000000
out[9] = -10.000000


In [ ]:
import numpy as np

# Hyper-parameters
HIDDEN_SIZE = 10
NUM_FEATURES = 10
SEQ_LEN = 10

# Helper functions
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh):
    """
    W_ih  : (4H, F)
    W_hh  : (4H, H)
    x_t   : (F,) column-vector
    h_prev: (H,)
    returns (4H,) pre-activation for gates
    """
    return W_ih @ x_t + W_hh @ h_prev + b_ih + b_hh

# LSTM cell for encoder
def lstm_cell_enc(x_t, h_prev, c_prev, W_ih, W_hh, b_ih, b_hh):
    """LSTM cell for encoder."""
    idx_i = slice(0, HIDDEN_SIZE)
    idx_f = slice(HIDDEN_SIZE, 2*HIDDEN_SIZE)
    idx_g = slice(2*HIDDEN_SIZE, 3*HIDDEN_SIZE)
    idx_o = slice(3*HIDDEN_SIZE, 4*HIDDEN_SIZE)

    gates = concat_mult_add(x_t, h_prev, W_ih, W_hh, b_ih, b_hh)

    i_raw = gates[idx_i]
    f_raw = gates[idx_f]
    g_raw = gates[idx_g]
    o_raw = gates[idx_o]

    i_t = sigmoid(i_raw)
    f_t = sigmoid(f_raw)
    g_t = np.tanh(g_raw)
    o_t = sigmoid(o_raw)

    c_t = f_t * c_prev + i_t * g_t
    h_t = o_t * np.tanh(c_t)

    def r(arr): return [round(v, 10) for v in arr.tolist()]
    print(f"\n{'='*60}\nEncoder LSTM Cell")
    print(f"x_t             : {r(x_t)}")
    print(f"h_prev          : {r(h_prev)}")
    print(f"c_prev          : {r(c_prev)}")
    print(f"i_raw (pre-sigm): {r(i_raw)}")
    print(f"f_raw (pre-sigm): {r(f_raw)}")
    print(f"g_raw (pre-tanh): {r(g_raw)}")
    print(f"o_raw (pre-sigm): {r(o_raw)}")
    print(f"i_t (sigmoid)   : {r(i_t)}")
    print(f"f_t (sigmoid)   : {r(f_t)}")
    print(f"g_t (tanh)      : {r(g_t)}")
    print(f"o_t (sigmoid)   : {r(o_t)}")
    print(f"c_t (cell state): {r(c_t)}")
    print(f"h_t (output)    : {r(h_t)}")

    return h_t, c_t

# Test case
if __name__ == "__main__":
    # Initialize test data (matching C++ test case)
    x_t = np.array([
        -0.0392750613, -0.0110346396, 0.0099107185, -0.0144685861,
        -0.0114205517, -0.0008815154, -0.0025537552, 0.0080107413,
        -0.0270011742, -0.0039597796
    ], dtype=np.float32)
    h_prev = np.zeros(HIDDEN_SIZE, dtype=np.float32)
    c_prev = np.zeros(HIDDEN_SIZE, dtype=np.float32)

    # Initialize weights and biases (matching C++ initialization)
    W_ih = np.full((4 * HIDDEN_SIZE, NUM_FEATURES), 0.1, dtype=np.float32)
    W_hh = np.full((4 * HIDDEN_SIZE, HIDDEN_SIZE), 0.05, dtype=np.float32)
    b_ih = np.zeros(4 * HIDDEN_SIZE, dtype=np.float32)
    b_hh = np.zeros(4 * HIDDEN_SIZE, dtype=np.float32)

    # Run LSTM cell
    h_t, c_t = lstm_cell_enc(x_t, h_prev, c_prev, W_ih, W_hh, b_ih, b_hh)


Encoder LSTM Cell
x_t             : [-0.0392750613, -0.0110346396, 0.0099107185, -0.0144685861, -0.0114205517, -0.0008815154, -0.0025537552, 0.0080107413, -0.0270011742, -0.0039597796]
h_prev          : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
c_prev          : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
i_raw (pre-sigm): [-0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736]
f_raw (pre-sigm): [-0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736]
g_raw (pre-tanh): [-0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736]
o_raw (pre-sigm): [-0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736, -0.00926736]
i_t (sigmoid)   : [0.4976831675, 0.4976831675, 0.4976831675, 0.4976831675, 0.497